Tarea 2: utilizar SageMaker Debugger

En este laboratorio, use Amazon SageMaker Debugger para analizar y detectar cuellos de botella, tasas de utilización de recursos y diversos problemas de entrenamiento durante los trabajos de entrenamiento, así como para recibir alertas sobre todo esto.

## Tarea 2.1: configuración del entorno

Instale los paquetes y las dependencias.

In [ ]:
%%capture

#install a compiler
%conda install -c conda-forge gcc -y

**Nota:** La instalación de GCC puede demorar hasta 5 minutos.

In [ ]:
#install-dependencies

%pip install pytest-cov
%pip install pytest-filter-subpackage
%pip install -U sagemaker
%pip install -U smdebug
%pip install -U shap

In [ ]:
#import libraries and set variable values

import sys
import boto3
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sagemaker
import sagemaker_datawrangler 
import shap

from mpl_toolkits.axes_grid1 import host_subplot

from sagemaker.debugger import (
    CollectionConfig,
    DebuggerHookConfig,
    FrameworkProfile,
    ProfilerConfig,
    ProfilerRule,
    Rule,
    rule_configs,
    TensorBoardOutputConfig
)

from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from sagemaker.xgboost.estimator import XGBoost
from smdebug.core import modes
from smdebug.trials import create_trial

base_job_name = "lab-7-smdebugger-job"
bucket = sagemaker.Session().default_bucket()
bucket_path = "s3://{}".format(bucket)
prefix = "lab-7-smdebugger"
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
save_interval = 5

A continuación, importe el conjunto de datos.

In [ ]:
#import-dataset
lab_test_data = pd.read_csv('adult_data_processed.csv')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.head()

Divida el conjunto de datos en conjuntos de datos de entrenamiento (70 %), validación (20 %) y prueba (10 %). Los conjuntos de datos de validación y entrenamiento se usan para crear el modelo en este laboratorio. No usará el conjunto de datos de pruebas en este laboratorio.

In [ ]:
#split-dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)

feature_names = list(train_data.columns)

Ahora, cargue el conjunto de datos a Amazon Simple Storage Service (Amazon S3).

In [ ]:
#upload-dataset
sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

## Tarea 2.2: modificar el script de entrenamiento para habilitar SageMaker Debugger

Debe modificar el script de entrenamiento que usó en el laboratorio anterior para guardar los tensores en un bucket de S3 de salida determinado, especificar qué tensores guardar y registrar los enlaces de depuración.

Para entrenar el modelo, deberá configurar lo siguiente:
- **Parámetros de enlaces de Debugger** para ajustar los intervalos de guardado de los tensores de salida en las fases de entrenamiento
- **Objeto de la regla de Debugger** para guardar los tensores de salida para la evaluación

En el caso de los *parámetros de enlaces de Debugger**, configure las colecciones de tensores incorporadas **metrics**, **feature_importance**, **full_shap** y **average_shap** para que se capturen durante el entrenamiento. Estas se configuran en **collection_configs** para **debugger_hook_config**. Las colecciones de tensores incorporadas **full_shap** y **average_shap** utilizan valores de Shapley (SHAP). SHAP explica una predicción de machine learning (ML) mediante la suposición de que el valor de cada función de una instancia de datos de entrenamiento es un jugador en un juego en el que la predicción es el monto. Luego, indica cómo distribuir el monto equitativamente entre las funciones. Consulte [Valores de referencia de SHAP para la explicabilidad](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html) para obtener más información sobre SHAP.

En el caso del **objeto de la regla de Debugger**, configure **rule_configs** en **rules** (reglas):
- [Informe del generador de perfiles](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-report.html#debugger-profiling-report): se ejecuta para detectar cuellos de botella del sistema y autogenera un informe de generación de perfiles.
- [Informe de entrenamiento con XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-report-xgboost.html): ejecuta un informe de XGBoost integral.
- [Sobreajuste](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#overfit): detecta si se está sobreajustando su modelo a los datos de entrenamiento mediante la comparación de la validación y las pérdidas de entrenamiento.
- [Sobreentrenamiento ](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#overtraining): detecta si se está sobreentrenando un modelo. Después de varias iteraciones del entrenamiento en un modelo con buen comportamiento (disminución de pérdida de entrenamiento y validación), el modelo se acerca a un mínimo de la función de pérdida y no mejora más. Si se sigue entrenando el modelo, la pérdida de validación podría empezar a aumentar porque el modelo comienza a sobreajustarse.
- [La perdida no disminuye](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#loss-not-decreasing): detecta si el valor de la pérdida no está disminuyendo a una tasa adecuada. Estas pérdidas deben ser escalares. 

In [ ]:
#enable-debugger
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)
# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    base_job_name=base_job_name,
    instance_count=1, 
    instance_type='ml.m5.4xlarge',
    #Set the hyperparameters
    hyperparameters= {
        "max_depth": "5",
        "eta": "0.2",
        "gamma": "4",
        "min_child_weight": "6",
        "subsample": "0.7",
        "objective": "binary:logistic",
        "num_round": "300",
    },
    sagemaker_session=sagemaker_session,
    max_run=1800,
    #Set the Debugger Hook Config
    debugger_hook_config=DebuggerHookConfig(
        s3_output_path=bucket_path,  # Required
        collection_configs=[  # For each of these, a new processing job will be run later in the lab
            CollectionConfig(name="metrics", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="feature_importance", parameters={"save_interval": str(save_interval)},),
            CollectionConfig(name="full_shap", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="average_shap", parameters={"save_interval": str(save_interval)}),
        ],
    ),
    #Set the Debugger Profiler Configuration
    profiler_config = ProfilerConfig(
        system_monitor_interval_millis=500,
        framework_profile_params=FrameworkProfile()

    ),
    #Configure the Debugger Rule Object
    rules = [
        ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
        Rule.sagemaker(rule_configs.create_xgboost_report()),  
        Rule.sagemaker(rule_configs.overfit()),
        Rule.sagemaker(rule_configs.overtraining()),
        Rule.sagemaker(rule_configs.loss_not_decreasing(),
            rule_parameters={
                "collection_names": "metrics",
                "num_steps": str(save_interval * 2),
            }
        )
    ]
)

## Tarea 2.3: ejecutar el trabajo de entrenamiento habilitado por Debugger

Ahora, entrene el modelo XGBoost usando el script habilitado por Debugger. La ejecución del entrenamiento demorará entre 5 y 10 minutos. Puede continuar con la próxima tarea mientras el trabajo de entrenamiento se está ejecutando y supervisar el avance del trabajo con SageMaker Debugger.

In [ ]:
#train-model
xgb.fit(
    inputs = data_inputs
)

## Tarea 2.4: supervisar el estado del trabajo de entrenamiento

En SageMaker Studio, puede revisar los componentes de prueba, incluidos todos los trabajos de SageMaker Debugger que inició. En este laboratorio, creó trabajos de lo siguiente: **informe de XGBoost**, **sobreajuste**, **sobreentrenamiento** y **pérdida que no disminuye**. Explórelos en SageMaker Studio.

El siguiente paso lo llevará a una nueva pestaña en SageMaker Studio. Para seguir esas instrucciones, utilice las siguientes opciones:
- **Opción 1**: ver las pestañas una al lado de la otra. Para crear una vista de pantalla dividida de la ventana principal de SageMaker Studio, arrastre la pestaña **lab_7.ipynb** hacia el costado o seleccione (con el botón derecho del mouse) la pestaña **lab_7.ipynb** y elija la opción **New View for Notebook** (Nueva vista para el cuaderno). Ahora puede tener las instrucciones a la vista mientras explora el grupo de funciones.
- **Opción 2**: alternar entre las pestañas de SageMaker Studio para seguir estas instrucciones.

1. Seleccione el ícono de **SageMaker Home** (Inicio de SageMaker).
2. Ejecute **Experiments** (Experimentos).

Verá la pestaña **Experiments** (Experimentos) en SageMaker Studio.

3. Seleccione **Unassigned runs** (Ejecuciones no asignadas).
4. En la lista, seleccione el **nombre** del trabajo cuyo tipo es **SageMakerTrainingJob**.

Se mostrarán los detalles del trabajo de entrenamiento.

5. En el lado izquierdo, seleccione la pestaña **Debugger** (Depurador).

SageMaker Debugger proporciona el estado de su trabajo de entrenamiento, que puede supervisar mientras se está ejecutando el entrenamiento de modelos. Cuando se complete, verá el estado de cualquier problema de entrenamiento especificado.

El análisis estará completo cuando todas las líneas de **estado** muestren el mensaje **No Issues Found** (No se encontraron problemas) o **Issues Found** (Se encontraron problemas). Las reglas de Debugger pueden demorar hasta 9 minutos en completarse.

Si se encuentran problemas, esto significa que debería solucionarlos en su modelo. ¿Se encontró algún problema para los trabajos? 

En este laboratorio, no resuelva ningún problema encontrado. Sin embargo, si desea resolverlos, puede abordarlos con una combinación de procesamiento del conjunto de datos y reentrenamiento del modelo con los hiperparámetros ajustados.

6. Cuando se complete el análisis, vuelva a la pestaña del cuaderno que tiene la etiqueta **lab_7.ipynb**.

## Tarea 2.5: realizar análisis posteriores al entrenamiento

Con SageMaker Debugger, puede crear registros del trabajo de procesamiento en Amazon CloudWatch, que puede usar para configurar alarmas personalizadas. Aquí, imprima la ubicación del lugar donde se almacenan los registros para cada métrica evaluada.

In [ ]:
#print-urls
def _get_rule_job_name(training_job_name, rule_configuration_name, rule_job_arn):
    """Helper function to get the rule job name with correct casing"""
    return "{}-{}-{}".format(
        training_job_name[:26], rule_configuration_name[:26], rule_job_arn[-8:]
    )


def _get_cw_url_for_rule_job(rule_job_name, region):
    return "https://{}.console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix".format(
        region, region, rule_job_name
    )


def get_rule_jobs_cw_urls(xgb):
    region = boto3.Session().region_name
    training_job = xgb.latest_training_job
    training_job_name = training_job.describe()["TrainingJobName"]
    rule_eval_statuses = training_job.describe()["DebugRuleEvaluationStatuses"]

    result = {}
    for status in rule_eval_statuses:
        if status.get("RuleEvaluationJobArn", None) is not None:
            rule_job_name = _get_rule_job_name(
                training_job_name, status["RuleConfigurationName"], status["RuleEvaluationJobArn"]
            )
            result[status["RuleConfigurationName"]] = _get_cw_url_for_rule_job(
                rule_job_name, region
            )
    return result


get_rule_jobs_cw_urls(xgb)

Las colecciones predeterminadas pueden recuperar tensores como ponderaciones, gradientes, sesgos y pérdidas que SageMaker Debugger crea a partir de su trabajo de entrenamiento, además de personalizar las colecciones usando los tensores. Genere una lista de los nombres y valores para los tensores guardados a fin de determinar cuáles trazar para el análisis más detallado.

In [ ]:
#retrieve-names
trial = create_trial(xgb.latest_job_debugger_artifacts_path())
trial.tensor_names()

In [ ]:
#retrieve-values
trial.tensor("average_shap/f1").values()

In [ ]:
#plot-tensors
shap_values = trial.tensor("full_shap/f10").value(trial.last_complete_step)
shap_no_base = shap_values[:, :-1]
shap_base_value = shap_values[0, -1]
shap.summary_plot(shap_no_base, plot_type="bar", feature_names=feature_names)

## Tarea 2.6: acceder al panel de información de SageMaker Debugger

1. Regrese a la pestaña **Experiments** (Experimentos).
2. Seleccione **Debugger** (Depurador), que se encuentra a la izquierda.
3. En la sección **Debugger insights** (Información de depurador), seleccione **lab-7-smdebugger-job** de la lista **Training job name** (Nombre del trabajo de entrenamiento).

SageMaker Studio abre una nueva pestaña **Insights** (Información) de Debugger para el trabajo y empieza a cargar datos.

SageMaker Debugger proporciona una descripción general del rendimiento del entrenamiento de su modelo en instancias de Amazon Elastic Compute Cloud (Amazon EC2). Explore SageMaker Debugger en SageMaker Studio y examine detalles contenidos en los informes.

La pestaña **Systems Metrics** (Métricas del sistema) incluye las siguientes secciones:

- **Resource utilization summary** (Resumen de utilización de los recursos)
- **CPU utilization summary** (Resumen de utilización de CPU)
- **GPU Utilization summary** (Resumen de utilización de GPU)

La pestaña **Rules** (Reglas) incluye la siguiente **información**: 

- **BatchSize**
- **LowGPUUtiliztion**
- **CPUBottleneck**
- **GPUMemoryIncrease**
- **StepOutlier**
- **MaxInitializationTime**
- **IOBottleneck**
- **LoadBalancing**

Los datos se propagarán en los gráficos y las tablas si se encuentra algún problema.

4. Puede descargar un informe del depurador si selecciona <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Download report</span> (Descargar informe) cerca de la parte superior de la pestaña **Debugger insights** (Información del depurador).

### Conclusión

¡Felicitaciones! Usó SageMaker Debugger para analizar, detectar y crear alertas para posibles problemas en el entrenamiento de modelos. Ahora puede usar la información generada a partir de sus informes y alertas para proporcionar información sobre el entrenamiento y la creación eficientes de un modelo más efectivo. El siguiente laboratorio se centra en el uso de SageMaker Clarify para detectar sesgos y proporcionar explicabilidad para las predicciones de modelos.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Conclusión**.